In [45]:
import sympy as sp

In [69]:
N = sp.vector.CoordSys3D('N')

In [70]:
class Node:
    def __init__(self, tup):
        x, y, z = tup
        self.position = x*N.i + y*N.j + z*N.k
        
    def __sub__(self, other):
        return self.position - other.position

In [104]:
class Force:
    def __init__(self, input1, input2):
        """
        input1 must be a starting point/node
        input2 can be another node to be an end point, or force vector
        """
        assert type(input1) == Node
        self.start_position = input1.position
        self.unique_symbols = set()
        
        if type(input2) == Node:
            diff_vector = input2 - input1
            self.force_vector = diff_vector / np.linalg.norm(diff_vector)
        elif type(input2) == sp.vector.vector.BaseVector:
            self.force_vector = input2
        else:
            fx, fy, fz = input2
            mx = my = mz = 1
            if type(fx) == tuple:
                mx, fx = fx
            if type(fx) == str:
                fx = sp.symbols(fx)
                self.unique_symbols.add(fx)
                
            if type(fy) == tuple:
                my, fy = fy
            if type(fy) == str:
                fy = sp.symbols(fy)
                self.unique_symbols.add(fy)
                
            if type(fz) == tuple:
                mz, fz = fz
            if type(fz) == str:
                fz = sp.symbols(fz)
                self.unique_symbols.add(fz)
                
            self.force_vector = mx*fx*N.i + my*fy*N.j + mz*fz*N.k
        self.torque_vector = sp.vector.cross(self.start_position, self.force_vector)

In [174]:
def solve_sum_zero(forces_eq_0, exclude_list):
    unique_symbols = set()
    sym_zero = 0*(N.i+N.j+N.k)
    force_eq_0 = 0*(N.i+N.j+N.k)
    torque_eq_0 = 0*(N.i+N.j+N.k)
    for i in forces_eq_0:
        unique_symbols |= i.unique_symbols
        force_eq_0 += i.force_vector
        torque_eq_0 += i.torque_vector
    for i in exclude_list:
        unique_symbols.remove(sp.symbols(i))
    symbols = list(unique_symbols)
    equations = []
    equations.append(sp.Eq(force_eq_0.components[N.i], 0))
    equations.append(sp.Eq(force_eq_0.components[N.j], 0))
    equations.append(sp.Eq(force_eq_0.components[N.k], 0))
    equations.append(sp.Eq(torque_eq_0.components[N.i], 0))
    equations.append(sp.Eq(torque_eq_0.components[N.j], 0))
    equations.append(sp.Eq(torque_eq_0.components[N.k], 0))
    solved = sp.solve(equations, symbols)
    for k in sorted(list(solved.keys()), key=lambda x: str(x)):
        display(sp.Eq(k, solved[k]))

In [143]:
from IPython.display import display

In [133]:
l = 1
a = 0.5
theta = np.deg2rad(30)
alpha = np.deg2rad(30)
mag_F_F = 10

In [79]:
A = Node((0, 0, -l))
B = Node((0, 0, l))
D = Node((-a*np.sin(theta), -a*np.cos(theta), 0))
E = Node((a, 0, 0))

In [170]:
F_A = Force(A, ('F_{Ax}', 'F_{Ay}', 'F_{Az}'))
F_B = Force(B, ('F_{Bx}', 'F_{By}', 0))
F_F_known = Force(D, (0, -sp.sin(alpha)*mag_F_F, -sp.cos(alpha) * mag_F_F))
F_F_unknown = Force(D, (0, (-sp.sin(alpha), 'F_F'),  (-sp.cos(alpha), 'F_F')))
F_P = Force(E, (0, 'F_P', 0))

In [172]:
solve_sum_zero([F_A, F_B, F_F_known, F_P])

Eq(F_P, -2.5)

Eq(F_{Ax}, -1.08253175473055)

Eq(F_{Ay}, 1.875)

Eq(F_{Az}, 8.66025403784439)

Eq(F_{Bx}, 1.08253175473055)

Eq(F_{By}, 5.625)

In [176]:
solve_sum_zero([F_A, F_B, F_F_unknown, F_P], ['F_F'])

Eq(F_P, -0.25*F_F)

Eq(F_{Ax}, -0.108253175473055*F_F)

Eq(F_{Ay}, 0.1875*F_F)

Eq(F_{Az}, 0.866025403784439*F_F)

Eq(F_{Bx}, 0.108253175473055*F_F)

Eq(F_{By}, 0.5625*F_F)